## Import Package & Global settings

In [1]:
import pandas as pd
import numpy as np
from glob import glob
import tensorflow as tf
from sklearn.model_selection import train_test_split
import plotly.express as px
# import emd
tf.keras.mixed_precision.set_global_policy('mixed_float16')

BATCH_SIZE = 32
CHANNEL_NUMBER = 3
WINDOW_SIZE = 200
SLIDING_STEP = int(WINDOW_SIZE * 0.25)
KEY_CLASS = {0:'undefined action', 1:'up', 2:'down', 3:'left', 4:'right', 5:'quick touch'}
CLASS_NUMBER = 5 # 0 is not a class
NUM_IMF = 3
LABEL_THRESHOLD = 0.8
BELIEF_THRESHOLD = 0.8
INITIAL_PULSE = 100 # abandon initial pulse data

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 3070 Ti Laptop GPU, compute capability 8.6


## Model architecture related definition

In [2]:
class PositionalEmbedding(tf.keras.layers.Layer):
    def __init__(self, vocab_size = 1024, d_model = 32):
        super().__init__()
        def positional_encoding(length, depth):
            depth = depth/2

            positions = np.arange(length)[:, np.newaxis]     # (seq, 1)
            depths = np.arange(depth)[np.newaxis, :]/depth   # (1, depth)

            angle_rates = 1 / (10000**depths)         # (1, depth)
            angle_rads = positions * angle_rates      # (pos, depth)

            pos_encoding = np.concatenate(
                [np.sin(angle_rads), np.cos(angle_rads)],
                axis=-1) 

            return pos_encoding
        
        self.d_model = d_model
        self.embedding = tf.keras.layers.Embedding(vocab_size, d_model, mask_zero=True) 
        self.pos_encoding = positional_encoding(2048, d_model)

    def compute_mask(self, *args, **kwargs):
        return self.embedding.compute_mask(*args, **kwargs)

    def call(self, x):
        batch_size = tf.shape(x)[0]
        x = tf.image.extract_patches(images=x,
                                    sizes=[1, CHANNEL_NUMBER, 2, x.shape[-1]],
                                    strides=[1, CHANNEL_NUMBER, 1, x.shape[-1]],
                                    rates=[1, 1, 1, 1],
                                    padding='VALID')
        patch_dims = x.shape[-1]
        x = tf.reshape(x, [batch_size, x.shape[1] * x.shape[2], patch_dims])
        x = self.embedding(x)
        # This factor sets the relative scale of the embedding and positional_encoding.
        x *= tf.math.sqrt(tf.cast(self.d_model, tf.float16))
        pe = self.pos_encoding[np.newaxis, np.newaxis, :patch_dims, :]
        for _ in range(x.shape[1] - 1):
            pe = np.concatenate([pe, self.pos_encoding[np.newaxis, np.newaxis, :patch_dims, :]], axis=1)
        x = x + tf.cast(pe, dtype=tf.float16)
        return x
    
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(EncoderLayer, self).__init__()
        
        def point_wise_feed_forward_network(d_model, dff):
            return tf.keras.Sequential([
                tf.keras.layers.Dense(dff, activation='elu'),  # (batch_size, seq_len, dff)
                tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
            ])

        self.mha = tf.keras.layers.MultiHeadAttention(num_heads = num_heads, key_dim = d_model)
        self.ffn = point_wise_feed_forward_network(d_model, dff)

        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, x, training = False):
        attn_output = self.mha(x, x)  # (batch_size, input_seq_len, d_model)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(x + attn_output)  # (batch_size, input_seq_len, d_model)

        ffn_output = self.ffn(out1)  # (batch_size, input_seq_len, d_model)
        ffn_output = self.dropout2(ffn_output, training=training)
        out2 = self.layernorm2(out1 + ffn_output)  # (batch_size, input_seq_len, d_model)

        return out2
    
class lrs(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=50):
        super().__init__()
        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)

        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)
    
    def get_config(self):
        config = {
            'd_model': self.d_model,
            'warmup_steps': self.warmup_steps,
        }
        return config

## Utils

In [3]:
def slicing(x, y):
    totalLength = x.shape[0]
    assert totalLength == y.shape[0], "Data numbers not matching with that of labels."
    if totalLength <= WINDOW_SIZE:
        return x, y

    y = one_hot(y)
    
    thresholdWindow = LABEL_THRESHOLD * WINDOW_SIZE
    retx = None
    rety = None
    retUnknown = None
    
    i = 0
    while (totalLength - i) > WINDOW_SIZE:
        new = (x[i:(i + WINDOW_SIZE), :])[np.newaxis, :]
        
        classSum = np.sum(y[i:(i + WINDOW_SIZE)], axis = 0)
        maxIdx = np.argmax(classSum)
        if classSum[maxIdx] > thresholdWindow:
            if not isinstance(retx, np.ndarray):
                retx = new.copy()
                rety = [maxIdx + 1]
            else:
                retx = np.concatenate([retx, new], axis=0)
                rety.append(maxIdx + 1)
        else:
            if not isinstance(retUnknown, np.ndarray):
                retUnknown = new.copy()
            else:
                retUnknown = np.concatenate([retUnknown, new], axis=0)

        i += SLIDING_STEP
        
    return np.transpose(retx[:, np.newaxis], (0, 3, 2, 1)), one_hot(rety), np.transpose(retUnknown[:, np.newaxis], (0, 3, 2, 1))

def one_hot(arr):
    ret = []
    for val in arr:
        tmp = [0] * CLASS_NUMBER
        if val > 0:
            tmp[val - 1] = 1
        ret.append(np.array(tmp))
        
    return np.array(ret)

def train_test_unknown_split(trainSignal, trainLabel, unknownActions, fold = None,  randomUnknown = True, base = 0.05, rand = 0.1):
    X_train = None
    X_test = None
    y_train = None
    y_test = None
    for x, y, u in zip(trainSignal, trainLabel, unknownActions):
        if (randomUnknown):
            if x.shape[0] // CLASS_NUMBER < u.shape[0]:
                X_unknown_add = u[np.random.choice(u.shape[0], size = int(x.shape[0] // CLASS_NUMBER), replace = False)]
                y_unknown_add = (np.random.rand(int(x.shape[0] // CLASS_NUMBER), CLASS_NUMBER) * rand + base)
            else:
                X_unknown_add = u
                y_unknown_add = (np.random.rand(u.shape[0], CLASS_NUMBER) * rand + base)

            Xt = np.concatenate([x, X_unknown_add], axis = 0)
            yt = np.concatenate([y, y_unknown_add], axis = 0)
            xtr, xte, ytr, yte = train_test_split(Xt, yt, test_size=0.2, random_state=343)
        else:
            xtr, xte, ytr, yte = train_test_split(x, y, test_size=0.2, random_state=343)
            
        if not isinstance(X_train, np.ndarray):
            X_train = xtr
            X_test = xte
            y_train = ytr
            y_test = yte
        else:
            X_train = np.concatenate([X_train, xtr], axis=0)
            X_test = np.concatenate([X_test, xte], axis=0)
            y_train = np.concatenate([y_train, ytr], axis=0)
            y_test = np.concatenate([y_test, yte], axis=0)
    
    return X_train, X_test, y_train, y_test

def evaluate(y, r, uw = 3, belief = BELIEF_THRESHOLD, v = True):
    gj, bj, bua = {}, {}, {}
    bu, tg, tb, lrconfusion, rlconfusion = 0, 0, 0, 0, 0

    for key, cls in KEY_CLASS.items():
        gj[cls] = 0
        bj[cls] = 0
        
    for r, p in zip(y, r.numpy()):
        rm = KEY_CLASS[np.argmax(r) + 1 if any(r) else 0]
        pm = KEY_CLASS[np.argmax(p) + 1 if p[np.argmax(p)] > belief else 0]
        if rm == pm:
            tg += 1
            gj[rm] += 1
        else:
            tb += 1
            bj[rm] += 1
            if rm == "undefined action":
                bu += 1
                if bua.get(pm, None):
                    bua[pm] += 1
                else:
                    bua[pm] = 1
            elif (rm == "left") and (pm == "right"):
                lrconfusion += 1
            elif (rm == "right") and (pm == "left"):
                rlconfusion += 1

    la, ra, ua = 0, 0, 0
    
    for key, cls in KEY_CLASS.items():
        if v:
            print("Action: {}, True: {}, False: {}, Accuracy: {:.4f}".format(cls, gj[cls], bj[cls], gj[cls] / (gj[cls] + bj[cls] + 0.001)))
        if cls == "left":
            la = gj[cls] / (gj[cls] + bj[cls] + 0.001)
            if v:
                print("Massive prediction error times: {}, portion: {:.4f}.".format(lrconfusion, lrconfusion / bj[cls]))
        elif cls == "right":
            ra = gj[cls] / (gj[cls] + bj[cls] + 0.001)
            if v:
                print("Massive prediction error times: {}, portion: {:.4f}.".format(rlconfusion, rlconfusion / bj[cls]))
        elif cls == "undefined action":
            ua = gj[cls] / (gj[cls] + bj[cls] + 0.001)
    if v:
        print("Total True: {}, False: {}, Accuracy: {:.4f}".format(tg, tb, tg / (tg + tb)))
        for cls, bp in bua.items():
            print("Action:{} ,bad prediction times: {}".format(cls, bp))

    return la + ra + uw * ua

def ConTradiction_model(inputShape, d_model = 32, convDropRate = 0.5, encDropRate = 0.7):
    input = tf.keras.layers.Input(shape = inputShape)
    conv = tf.keras.layers.Conv2D(d_model, (1, int(WINDOW_SIZE * 0.5 // 3)), padding='same', activation='elu',
                            kernel_constraint=tf.keras.constraints.max_norm(0.25))(input)
    bnorm = tf.keras.layers.BatchNormalization()(conv)
    pooling = tf.keras.layers.AveragePooling2D((1, 8), padding='same')(bnorm)
    drop = tf.keras.layers.Dropout(convDropRate)(pooling)
    conv2 = tf.keras.layers.Conv2D(d_model, (1, int(WINDOW_SIZE * 0.5 // 6)), padding='same', activation='elu',
                            kernel_constraint=tf.keras.constraints.max_norm(0.25))(drop)
    bnorm2 = tf.keras.layers.BatchNormalization()(conv2)
    pooling2 = tf.keras.layers.AveragePooling2D((1, 4), padding='same')(bnorm2)
    drop2 = tf.keras.layers.Dropout(convDropRate)(pooling2)

    #transformer encoder
    encoder = EncoderLayer(d_model, 8, 2 * d_model, encDropRate)(drop2)
    #Classification
    flatten = tf.keras.layers.Flatten()(encoder)
    output = tf.keras.layers.Dense(CLASS_NUMBER, activation='softmax')(flatten)
    model = tf.keras.Model(inputs=input, outputs=output)
    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=lrs(d_model, 50)),
                    loss = tf.keras.losses.CategoricalCrossentropy(),
                    metrics = [tf.keras.metrics.CategoricalAccuracy()])
    return model

## Load, preprocess and split record files

In [4]:
trainSignalFiles = glob(".\\data\\*_record_X.npy")
trainLabelFiles = [x.replace('X', 'y') for x in trainSignalFiles]

numX = 0
numUX = 0
sigPLot = None
trainSignal, trainLabel, unknownActions = [], [], []
for sfp, lfp in zip(trainSignalFiles, trainLabelFiles):
    print("loaded {} and {}.".format(sfp, lfp))
    tempSig = np.load(sfp)[INITIAL_PULSE:]
    tempLbl = np.load(lfp)[INITIAL_PULSE:]
    sigPLot = tempSig if not isinstance(sigPLot, np.ndarray) else np.concatenate([sigPLot, tempSig], axis=0)
    X, y, X_unknown = slicing(tempSig, tempLbl)
    trainSignal.append(X)
    trainLabel.append(y)
    unknownActions.append(X_unknown)
    numX += X.shape[0]
    numUX += X_unknown.shape[0]

print("Number of X: {}, unknown X: {}".format(numX, numUX))

X_train, X_test, y_train, y_test = train_test_unknown_split(trainSignal, trainLabel, unknownActions, 
                                                            randomUnknown = True, base = 0.15, rand = 0.1)

loaded .\data\2023_Mar_11_224807_l5m6r7_record_X.npy and .\data\2023_Mar_11_224807_l5m6r7_record_y.npy.
loaded .\data\2023_Mar_12_015657_l5m6r7_record_X.npy and .\data\2023_Mar_12_015657_l5m6r7_record_y.npy.
loaded .\data\2023_Mar_14_153445_l5m6r7_record_X.npy and .\data\2023_Mar_14_153445_l5m6r7_record_y.npy.
loaded .\data\2023_Mar_14_170710_l5m6r7_record_X.npy and .\data\2023_Mar_14_170710_l5m6r7_record_y.npy.
loaded .\data\2023_Mar_21_182116_l5m6r7_record_X.npy and .\data\2023_Mar_21_182116_l5m6r7_record_y.npy.
loaded .\data\2023_Mar_21_185848_l5m6r7_record_X.npy and .\data\2023_Mar_21_185848_l5m6r7_record_y.npy.
loaded .\data\2023_Mar_21_190722_l5m6r7_record_X.npy and .\data\2023_Mar_21_190722_l5m6r7_record_y.npy.
loaded .\data\2023_Mar_29_170717_l5m6r7_record_X.npy and .\data\2023_Mar_29_170717_l5m6r7_record_y.npy.
loaded .\data\2023_Mar_29_191038_l5m6r7_record_X.npy and .\data\2023_Mar_29_191038_l5m6r7_record_y.npy.
loaded .\data\2023_Mar_29_203224_l5m6r7_record_X.npy and .\data\

## Train model

In [54]:
model = ConTradiction_model((CHANNEL_NUMBER, WINDOW_SIZE, 1), convDropRate=0.4, encDropRate=0.4)
model.summary()
history = model.fit(x=X_train,
                    y=y_train,
                    batch_size=BATCH_SIZE,
                    epochs=500,
                    validation_data=[X_test, y_test])

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 3, 200, 1)]       0         
                                                                 
 conv2d_14 (Conv2D)          (None, 3, 200, 32)        1088      
                                                                 
 batch_normalization_14 (Bat  (None, 3, 200, 32)       128       
 chNormalization)                                                
                                                                 
 average_pooling2d_14 (Avera  (None, 3, 25, 32)        0         
 gePooling2D)                                                    
                                                                 
 dropout_38 (Dropout)        (None, 3, 25, 32)         0         
                                                                 
 conv2d_15 (Conv2D)          (None, 3, 25, 32)         1641

In [56]:
history = model.fit(x=X_train,
                    y=y_train,
                    batch_size=BATCH_SIZE,
                    epochs=200,
                    validation_data=[X_test, y_test])

Epoch 1/200
184/184 [==============================] - 2s 9ms/step - loss: 0.6932 - categorical_accuracy: 0.7365 - val_loss: 0.7714 - val_categorical_accuracy: 0.7023
Epoch 2/200
184/184 [==============================] - 1s 8ms/step - loss: 0.6900 - categorical_accuracy: 0.7364 - val_loss: 0.7756 - val_categorical_accuracy: 0.7030
Epoch 3/200
184/184 [==============================] - 1s 7ms/step - loss: 0.6840 - categorical_accuracy: 0.7449 - val_loss: 0.7982 - val_categorical_accuracy: 0.6976
Epoch 4/200
184/184 [==============================] - 1s 8ms/step - loss: 0.6879 - categorical_accuracy: 0.7408 - val_loss: 0.7688 - val_categorical_accuracy: 0.7192
Epoch 5/200
184/184 [==============================] - 1s 8ms/step - loss: 0.6899 - categorical_accuracy: 0.7355 - val_loss: 0.7665 - val_categorical_accuracy: 0.7165
Epoch 6/200
184/184 [==============================] - 1s 8ms/step - loss: 0.6813 - categorical_accuracy: 0.7460 - val_loss: 0.7384 - val_categorical_accuracy: 0.713

In [68]:
f1 = np.array(history.history['loss']).flatten()
valf1 = np.array(history.history['val_loss']).flatten()
px.line(pd.DataFrame(np.array([f1, valf1]).T, columns=['loss', 'val_loss'])).show()

In [69]:
model.save_weights("./model/MTJaw0404_700_W200_T8799/", save_format="tf")

## Evaluation

In [58]:
X_unknown = None
for u in unknownActions:
    if not isinstance(X_unknown, np.ndarray):
        X_unknown = u
    else:
        X_unknown = np.concatenate([X_unknown, u], axis=0)
All_X = np.concatenate([X_test, X_unknown])
All_y = np.concatenate([y_test, np.array([[0] * CLASS_NUMBER for _ in range(X_unknown.shape[0])])])

In [59]:
res = model(All_X)

In [67]:
evaluate(All_y, res, belief = 0.8799)

Action: undefined action, True: 6604, False: 1170, Accuracy: 0.8495
Action: up, True: 64, False: 60, Accuracy: 0.5161
Action: down, True: 13, False: 89, Accuracy: 0.1274
Action: left, True: 345, False: 335, Accuracy: 0.5074
Massive prediction error times: 24, portion: 0.0716.
Action: right, True: 277, False: 250, Accuracy: 0.5256
Massive prediction error times: 7, portion: 0.0280.
Action: quick touch, True: 0, False: 45, Accuracy: 0.0000
Total True: 7303, False: 1949, Accuracy: 0.7893
Action:left ,bad prediction times: 421
Action:up ,bad prediction times: 157
Action:right ,bad prediction times: 571
Action:down ,bad prediction times: 21


3.58146255144404

In [66]:
#Grid search
parameterSelection = []
lb, ub = 0, 0.9999
best = None
for n in range(3):
    for i in range(10):
        threshold = lb + (i + 1) * 0.1 * (ub - lb)
        parameterSelection.append([threshold, evaluate(All_y, res, uw = 3, belief = threshold, v = False)])
    parameterSelection.sort(key=lambda x:x[1])
    lb, ub = parameterSelection[-2][0], parameterSelection[-1][0]
    best = parameterSelection[-1]
    print("iteration", n, ":", best, lb, ub)

iteration 0 : [0.89991, 3.577545373259265] 0.7999200000000001 0.89991
iteration 1 : [0.879912, 3.58146255144404] 0.869913 0.879912
iteration 2 : [0.879912, 3.58146255144404] 0.879912 0.879912


In [51]:
test = ConTradiction_model((CHANNEL_NUMBER, WINDOW_SIZE, 1))
test.load_weights('./model/' + "MTJaw0404_700_W200_T8799" + '/')
res = test(All_X)

ValueError: Received incompatible tensor with shape (1, 33, 1, 32) when attempting to restore variable with shape (1, 41, 1, 32) and name layer_with_weights-0/kernel/.ATTRIBUTES/VARIABLE_VALUE.

In [ ]:
px.line(np.load('./data/2023_Mar_21_184531_l5m6r7_record_X.npy')[100:])

In [ ]:
px.line(np.load('./data/2023_Mar_21_184531_l5m6r7_record_y.npy')[100:])

In [37]:
X, y, u = slicing(np.load('./data/2023_Mar_30_182133_l5m6r7_record_Empty_X.npy'), np.load('./data/2023_Mar_30_182133_l5m6r7_record_Empty_y.npy'))

TypeError: 'NoneType' object is not subscriptable

In [ ]:
r = test(np.concatenate([X, u]))

In [ ]:
evaluate(np.concatenate([y, np.zeros((u.shape[0], CLASS_NUMBER))]), r, belief = 0.8969)